<a href="https://colab.research.google.com/github/Mindgaze/Gerador-de-Fus-es/blob/main/Create_an_python_codd_deployable_on_google_colab_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Of course. Here is a complete Python script designed for Google Colab that sets up an AI image art generator. This code allows you to fuse two images based on a text prompt and generates a new image in a 9:16 aspect ratio with a full-body view.

Just copy and paste each code block into a separate cell in your Google Colab notebook and run them in order.

-----

## 1\. Setup Environment ⚙️

This first step installs all the necessary libraries. It might take a few minutes to complete.

In [1]:
# Install required Python libraries
!pip install diffusers transformers accelerate invisible_watermark -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.9 MB/s eta 0:00:00


-----

## 2\. Load the AI Model 🧠

Next, we'll load the pre-trained Stable Diffusion model. This model is powerful and can create high-quality images. This step downloads the model weights, so it can also take a moment.

In [2]:
import torch
from diffusers import StableDiffusionImg2ImgPipeline
from PIL import Image
import requests
from io import BytesIO

# Load the pre-trained image-to-image pipeline
# Using a specific model known for good artistic results
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionImg2ImgPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    use_safetensors=True
)

# Move the model to the GPU for faster processing
pipe = pipe.to("cuda")

print("✅ Model loaded successfully!")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

scheduler_config.json:   0%|          | 0.00/308 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

...

unet/diffusion_pytorch_model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

config.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

vae/diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

-----

## 3\. Upload Your Images 🖼️

Run this cell to upload the two images you want to fuse. An "upload" button will appear.

  * **Image 1 (Style Image):** This image will provide the initial structure and composition for the new artwork.
  * **Image 2 (Content Image):** While not directly used as a second input image in this pipeline, its concepts should be described in your prompt. The AI will fuse the *idea* of this image with the structure of the first one.

<!-- end list -->

In [ ]:
from google.colab import files

print("Please upload your first image (the base/style image)...")
uploaded_style = files.upload()

if uploaded_style:
    style_image_name = next(iter(uploaded_style))
    print(f"\nUploaded '{style_image_name}' as the style image.")
else:
    print("No style image was uploaded.")

-----

## 4\. Generate Your AI Art ✨

This is the final step where the magic happens.

1.  **Define your prompt:** Clearly describe the final image you want. Mention elements from both of your uploaded images and add "Full Length Body Size View" to ensure the framing is correct.
2.  **Run the cell:** The AI will use your first image as a base and transform it according to your prompt, generating a new piece of art in the specified 9:16 aspect ratio.

<!-- end list -->

In [ ]:
# --- Your Settings ---
prompt = "A fusion of the two images, a surreal portrait of a knight wearing a floral helmet, cinematic lighting, dramatic, Full Length Body Size View" #@param {type:"string"}

# --- Image Generation ---
if uploaded_style:
    # Open the uploaded image
    init_image = Image.open(BytesIO(uploaded_style[style_image_name])).convert("RGB")

    # Define the 9:16 aspect ratio dimensions
    # These are standard resolutions for this ratio
    width = 576
    height = 1024

    # Resize the initial image to match the target output dimensions
    init_image = init_image.resize((width, height))

    print("⏳ Generating your image... this may take a moment.")

    # Generate the image
    # 'strength' controls how much the original image is changed (0.0-1.0)
    # 'guidance_scale' controls how closely the prompt is followed (higher is stricter)
    generator = torch.Generator(device="cuda").manual_seed(1024) # for reproducible results
    image = pipe(
        prompt=prompt,
        image=init_image,
        strength=0.75,
        guidance_scale=7.5,
        generator=generator
    ).images[0]

    print("\n🎉 Generation complete! Here is your fused artwork:")
    display(image)

else:
    print("⚠️ Please upload a style image in the previous cell before running this one.")